In [1]:
import os
import model_ed_tcn as md
import numpy as np

Using TensorFlow backend.


In [2]:
def load_data(datapath, numOfSubjects):

    ext = '.npy'
    
    x_data = []
    y_data = []
    
    for index in range(numOfSubjects):
        x = np.load(os.path.join(datapath, 'x_data' + str(index) + ext))
        y = np.load(os.path.join(datapath, 'y_data' + str(index) + ext))
        
        x_data.append(x)
        y_data.append(y)
    
    

        
        
    return x_data, y_data

In [3]:
x_data, y_data = load_data('../data', 36)

In [4]:
model_obj = md.ED_TCN_fmap_fuse(
    n_classes=6,
    input_length=128,
    early_fuse_cnn_channels=64,
    tcn_encoder_channels=[64]*3,
    tcn_decoder_channels=[64]*2,
    tcn_encoder_maxpool_size=[2]*3,
    tcn_decoder_upsample_size=[2, 2],
    early_fuse_conv_kernel_size=3,
    before_encode_pool_size=2,
    tcn_kernel_size=5,
    tcn_drop_rate=0.3,
    use_spatial_drop=True,
    drop_rate_before_midfc=0.5,
    drop_rate_before_fcclassifier=0.5,
    use_init_batch_norm=True,
)

In [5]:
def create_model(learning_rate=0.001):
    global model_obj
    
    model = model_obj.create_model()
    
    import tensorflow
    model.compile(optimizer=tensorflow.keras.optimizers.Adam(lr=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])
    
    return model

In [6]:
model = create_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 3)            0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 128, 3)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 3)            12        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 128, 64)           640       
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 64)           256       
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 128, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 64)            0   

In [7]:
# Tunning hyper parameter
learning_rates = [1e-3, 1e-4, 1e-5]
batch_sizes = [16, 32, 64]


In [ ]:
train_results = []
num_of_subjects = 36

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        count = 0
        sum = 0
        
        print('Train with learning rate ' + str(learning_rate) + ' and batch size ' + str(batch_size))
        
        for index in range(num_of_subjects):
                
            x_valid = x_data[index]
            y_valid = y_data[index]

            x_train_list = x_data[:index] + x_data[index + 1:]
            y_train_list = y_data[:index] + y_data[index + 1:]

            x_train = np.concatenate(tuple(x_train_list))
            y_train = np.concatenate(tuple(y_train_list))


            model = create_model(learning_rate)
            

            print('Training with valid set: ' + str(index))

            H = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), batch_size=batch_size, epochs=20, verbose=1)

            best_acc = np.max(H.history['val_accuracy'])
            print('Best acc of this valid set ' + str(best_acc))

            count += y_valid.shape[0]
            sum += best_acc * y_valid.shape[0]
        
        
        train_results.append((learning_rate, batch_size, sum / count))
        print('Mean Acc : ' + str(sum / count))
        print('-----------------------------------------------------------')
        

Train with learning rate 0.001 and batch size 16
Training with valid set: 0
Train on 16304 samples, validate on 228 samples
Epoch 1/20
16304/16304 [==============================] - 39s 2ms/step - loss: 0.6050 - accuracy: 0.7883 - val_loss: 0.3296 - val_accuracy: 0.8991
Epoch 2/20
16304/16304 [==============================] - 10s 583us/step - loss: 0.3475 - accuracy: 0.8917 - val_loss: 0.2508 - val_accuracy: 0.9035
Epoch 3/20
16304/16304 [==============================] - 10s 585us/step - loss: 0.2656 - accuracy: 0.9201 - val_loss: 0.1666 - val_accuracy: 0.9035
Epoch 4/20
16304/16304 [==============================] - 9s 582us/step - loss: 0.2276 - accuracy: 0.9354 - val_loss: 0.2167 - val_accuracy: 0.9123
Epoch 5/20
16304/16304 [==============================] - 10s 586us/step - loss: 0.2100 - accuracy: 0.9398 - val_loss: 0.2302 - val_accuracy: 0.9035
Epoch 6/20
16304/16304 [==============================] - 10s 585us/step - loss: 0.1782 - accuracy: 0.9501 - val_loss: 0.1438 - val_ac

In [ ]:
f = open('result.txt', 'w')
for e in train_results:
    f.write(str(e))
f.close()